In [25]:
import requests
import pandas as pd
from household_package.clean import clean_data
from google.cloud import storage
import os
import pickle

In [57]:
user_input = {
  "TYPEHUQ": 1,
  "NHSLDMEM": 1,
  "state_name": "Alabama",
  "REGIONC": "SOUTH",
  "BA_climate": "Hot-Humid",
  "SQFTEST": 240,
  "STORIES": 1,
  "YEARMADERANGE": 1,
  "NCOMBATH": 0,
  "NHAFBATH": 0,
  "TOTROOMS": 1,
  "WALLTYPE": 1,
  "ROOFTYPE": 1,
  "WINDOWS": 1,
  "SWIMPOOL": 1,
  "SOLAR": 1,
  "SMARTMETER": 1,
  "TELLWORK": 1,
  "DESKTOP": 0,
  "NUMLAPTOP": 0,
  "TVCOLOR": 0,
  "DISHWASH": 1,
  "MICRO": 0,
  "NUMFRIG": 0,
  "CWASHER": 1,
  "DRYER": 1,
  "LGTIN1TO4": 0,
  "LGTIN4TO8": 0,
  "LGTINMORE8": 0,
  "AIRCOND": 1,
  "EQUIPM": 3,
  "HEATHOME": 1,
  "NUMPORTEL": 0
}

url = "http://localhost:8000/predict"

response = requests.get(url, user_input)
print(response.content)

b'{"TYPEHUQ":1,"NHSLDMEM":1,"state_name":"Alabama","REGIONC":"SOUTH","BA_climate":"Hot-Humid","SQFTEST":240,"STORIES":1,"YEARMADERANGE":1,"NCOMBATH":0,"NHAFBATH":0,"TOTROOMS":1,"WALLTYPE":1,"ROOFTYPE":1,"WINDOWS":1,"SWIMPOOL":1,"SOLAR":1,"SMARTMETER":1,"TELLWORK":1,"DESKTOP":0,"NUMLAPTOP":0,"TVCOLOR":0,"DISHWASH":1,"MICRO":0,"NUMFRIG":0,"CWASHER":1,"DRYER":1,"LGTIN1TO4":0,"LGTIN4TO8":0,"LGTINMORE8":0,"AIRCOND":1,"EQUIPM":3,"HEATHOME":1,"NUMPORTEL":0}'


In [61]:
X_new = pd.DataFrame(response.json(), index=[0])
X_new = clean_data(X_new)

In [59]:
X_new

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,TYPEHUQ,NHSLDMEM,state_name,REGIONC,BA_climate,SQFTEST,...,MICRO,NUMFRIG,CWASHER,DRYER,AIRCOND,EQUIPM,HEATHOME,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,1,0,1,1,Mobile,1,Alabama,SOUTH,Hot-Humid,240,...,0,0,1,1,1,Furnace,1,0,0,0


In [49]:
BUCKET_NAME="01_household_energy"
LOCAL_REGISTRY_PATH = "../model_h5/loaded/"

In [50]:
client = storage.Client()
blobs = list(client.get_bucket(BUCKET_NAME).list_blobs(prefix="model"))

In [51]:
latest_blob = max(blobs, key=lambda x: x.updated)
#print(latest_blob.name)
latest_model_path_to_save = os.path.join(LOCAL_REGISTRY_PATH, latest_blob.name)
print(latest_model_path_to_save)
latest_blob.download_to_filename(latest_model_path_to_save)
# for sklearn (baseline)
with open(latest_model_path_to_save , 'rb') as f:
    latest_model = pickle.load(f)

../model_h5/loaded/models/baseline_20231130-102940.pkl


In [52]:
latest_model.predict(X_new)[0]

11040.0